In [ ]:
#As a first step, the program extracts information about type of session from each folder.
## The program reads session_metadata.json file from each folder and renames the corresponding folders by adding information about type of session
## For instance, folder session_1 will be renamed as session_1_GAME

import glob
import json
import os, stat
 
# Specify path of the dataset
json_session_files = glob.glob(r'D:\Jay\brooklyn-mic\**\session_metadata.json',recursive=True)

for file in json_session_files:
    file_name=os.path.abspath(file)
    file_path=os.path.dirname(file_name)
    os.chmod(file_name ,stat.S_IWRITE)
    f = open(file)
    data = json.load(f)
    for key in data:
        if (data[key] == 'GAME'):
            new_foldername=file_path+'_GAME'
            os.rename(file_path,new_foldername)
        elif (data[key] == 'CONV'):
            new_foldername=file_path+'_CONV'
            os.rename(file_path,new_foldername)
        elif (data[key] == 'WOZ'):
            new_foldername=file_path+'_WOZ'
            os.rename(file_path,new_foldername)

In [ ]:
#As a second step, this program reads all the ipu_data.csv file from each session and combines all the files into one.
## ipu_data file comprises of IPU segmentations of the conversation. (See READ.md in the dataset)

import pandas as pd
import glob, os
import json

#Specify path of the dataset
files = glob.glob(r'D:\Jay\brooklyn-mic\**\ipu_data.csv',recursive=True)

#Specify output path with file name to save the merged ipu_data
out_name=r'D:\Jay\brooklyn-mic\merged.csv'

df = pd.concat([pd.read_csv(fp).assign(File_path=(fp)) for fp in files])

def get_conversational_group(text):
    if "CONV" in text:
        return 'CONV'
    elif "GAME" in text:
        return 'GAME'
    elif "WOZ" in text:
        return 'WOZ'
    else:
        return 'NONE'

df['Conversation_type'] = df['File_path'].apply(get_conversational_group)
df['Session_path'] = df['File_path'] 
df['Session_path'] = df['Session_path'].str.replace(r'\b(task.*)', 'session_metadata.json')

df=df.assign(lambda x: pd.Series(json.loads(df['Session_path'])), axis=1, result_type='expand')

df.to_csv(out_name, index=False,header=True,sep='\t')

In [24]:
#As a final step, this program reads each session_metadata.json from each folder and adds the information to the output file obtained in second step.
## The program reads session_metadata.json file from each folder and adds the information to merged ipu_data file

import pandas as pd
import json

#Add the path of file name obtained in the second step
file_name=r'D:\Jay\brooklyn-mic\merged.csv'

#Add the output path of the file name
out_name=r'D:\Jay\brooklyn-mic\merged_session.csv'

#The following qualitative information will be added from each session_metadata.json file
##question, spk_a_likeable, spk_a_smoothness, spk_b_likeable, spk_b_smoothness, spk_id_a, spk_id_b, type
df1=pd.DataFrame(columns=['question','spk_a_likeable','spk_a_smoothness','spk_b_likeable','spk_b_smoothness','spk_id_a','spk_id_b','type'])

csv_input = pd.read_csv(file_name,delimiter='\t')

for index, row in csv_input.iterrows():
     with open(row['Session_path'], 'r') as datafile:
          data = json.load(datafile)
     t1 = pd.DataFrame.from_dict(data, orient='index')
     t2= t1.transpose()
     df1=pd.concat([df1, t2]).reset_index(drop=True)
df3=pd.concat([csv_input, df1], axis=1)
df3.to_csv(out_name, index=False,header=True,sep='\t')